In [1]:
import boto3

In [7]:
# Submit a batch job
def submit_job(job_name, job_queue, job_definition, container_overrides, parameters):
    client = boto3.client('batch')
    response = client.submit_job(
        jobName=job_name,
        jobQueue=job_queue,
        jobDefinition=job_definition,
        containerOverrides=container_overrides,
        parameters=parameters,
        schedulingPriorityOverride=99,
        shareIdentifier="alex",
        retryStrategy={
            'attempts': 1
        }
    )
    return response['jobId']

# Submit an array job
def submit_array_job(job_name, job_queue, job_definition, container_overrides, array_properties):
    client = boto3.client('batch')
    response = client.submit_job(
        jobName=job_name,
        jobQueue=job_queue,
        jobDefinition=job_definition,
        containerOverrides=container_overrides,
        arrayProperties=array_properties,
        schedulingPriorityOverride=99,
        shareIdentifier="alex",
    )
    return response['jobId']

# Get the status of a job
def get_job_status(job_id):
    client = boto3.client('batch')
    response = client.describe_jobs(
        jobs=[job_id]
    )
    return response['jobs'][0]['status']

def get_cloudwatch_logs(job_id):
    client = boto3.client('batch')
    response = client.describe_jobs(
        jobs=[job_id]
    )
    log_stream_name = response['jobs'][0]['container']['logStreamName']

    logs_client = boto3.client('logs')
    log_group_name = f'/aws/batch/auspatious-ldn'

    response = logs_client.get_log_events(
        logGroupName=log_group_name,
        logStreamName=log_stream_name,
        startFromHead=True
    )

    events = response['events']
    for event in events:
        print(event['message'])

In [30]:
job_name = "real-job"
job_queue = "Fargate"
job_definition = "auspatious-ldn"
# job_command = ['aws', 's3', 'ls', 's3://data.ldn.auspatious.com/']
container_overrides = {
    "command": [
        "ldn-processor",
        "--tile",
        "Ref::tile",
        "--year",
        "Ref::year",
        "--version",
        "Ref::version",
        "Ref::extra_options",
    ],
    "resourceRequirements": [
        {"type": "VCPU", "value": "16"},
        {"type": "MEMORY", "value": "122880"},
    ],
}
parameters = {
    "tile": "238,47",
    "year": "2023",
    "version": "0.1.0",
    "extra_options": "--overwrite",
}

job_id = submit_job(
    job_name, job_queue, job_definition, container_overrides, parameters
)
print("Job submitted with id:", job_id)

status = get_job_status(job_id)
print("Job status:", status)

Job submitted with id: 5bf65070-8855-4513-b220-e20bfcf2d5c5
Job status: SUBMITTED


In [34]:
# Get status of job every 10 seconds
while status not in('SUCCEEDED', 'FAILED'):
    new_status = get_job_status(job_id)
    if new_status != status:
        print('Job status:', status)
        status = new_status

    import time
    time.sleep(10)
else:
    print(f'Job {status} successfully')

Job status: RUNNABLE
Job status: STARTING
Job status: RUNNING
Job status: RUNNABLE
Job status: STARTING


KeyboardInterrupt: 

In [37]:
get_cloudwatch_logs(job_id)

2024-10-14 08:12:50 INFO (238_47): Starting full run...
2024-10-14 08:12:52 INFO (238_47): Found 186 items
2024-10-14 08:12:52 INFO (238_47): Loading 186 items
2024-10-14 08:12:54 INFO (238_47): Loaded data with shape lon: 5000 lat: 5000 time: 98
2024-10-14 08:12:54 INFO (238_47): Preparing data...
2024-10-14 08:12:54 INFO (238_47): Resampling to monthly...
2024-10-14 08:12:55 INFO (238_47): Interpolating and filling gaps...
/code/.venv/lib/python3.12/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(
/code/.venv/lib/python3.12/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(
/code/.venv/lib/python3.12/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject

In [ ]:
--overwrite

In [ ]:
job_name = 'test-job'
job_queue = 'LowPriorityFargate'
job_definition = 'auspatious-ldn'
# job_command = ['aws', 's3', 'ls', 's3://data.ldn.auspatious.com/']
job_command = ["echo", 'Ref::test']
array_properties = {
    'size': 2
}
job_id = submit_array_job(job_name, job_queue, job_definition, job_command, array_properties=array_properties)
print('Job submitted with id:', job_id)

status = get_job_status(job_id)
print('Job status:', status)

# Get status of job every 10 seconds
while status != 'SUCCEEDED':
    status = get_job_status(job_id)
    import time
    time.sleep(10)
else:
    print('Job completed successfully')